In [ ]:
def detect_faces(image_path):
    # Load YOLO model weights and configuration
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    
    # Load COCO class names
    classes = []
    with open("coco.names", "r") as f:
        classes = f.read().strip().split("\n")
    
    # Get YOLO layer names and output layers
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    
    # Read and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape
    
    # Create a blob from the image
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    
    # Set the input for the YOLO network
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    # Process YOLO outputs to get bounding boxes and class IDs
    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Minimum confidence threshold
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
    
    # Apply Non-Maximum Suppression to filter out overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    # Iterate through detections and extract faces
    extracted_faces = []
    for i in indices:
        i = i[0]
        box = boxes[i]
        x, y, w, h = box
        face = img[y:y+h, x:x+w]
        extracted_faces.append(face)
    
    return extracted_faces